#### Scenario generator

In [1]:
def get_scenario(
        tot_area=24, 
        res=0.2, 
        people_range=(2, 5),
        p_per_group=(1,2),
        area_dim=2,
        min_dist=1,
        group_range=(0,1)        
        ):
    """
    """
    goal, q_people, q_obs, q_area = get_random_queue(tot_area, res, people_range)
    t_people, t_obs, t_area = get_random_talking_people(tot_area, res, p_per_group, area_dim, min_dist, [q_area], group_range)
    p_people = get_random_passing_by(tot_area, res, limit_area=t_area)

    if len(q_obs) > 0:
        obs = np.concatenate((q_obs, t_obs),axis=0)
    else:
        obs = t_obs
    
    return goal, q_people, t_people, p_people, obs

In [2]:
def get_state(goal, q_people, t_people, p_people, tot_area, res):
    """
    """
    m = int(tot_area/res)
    l = int(m/2)
    img = np.ones((m,m,3), dtype=np.uint8)*255
    for person in q_people:
        img[person[1]+l, person[0]+l, 0] = 0

    for person in t_people:
        img[person[1]+l, person[0]+l, 0] = 0

    for person in p_people:
        img[person[1]+l, person[0]+l, 0] = 0        

    img[goal[1]+l, goal[0]+l, 1] = 0
    
    return np.array(img, dtype=np.uint8)

In [3]:
def get_label(obs, tot_area, res):
    """
    """
    m = int(tot_area/res)
    l = int(m/2)
    img = np.zeros((m,m,1))
    for ob in obs:
        img[ob[1]+l, ob[0]+l] = [1]

    return np.array(img, dtype=np.uint8)

#### Gather data

In [4]:
def gather_single_data(tot_area, res, people_range, p_per_group, area_dim, min_dist, group_range):
    """
    """
    data = []
    goal, q_people, t_people, p_people, obs = get_scenario(tot_area, res, people_range, p_per_group, area_dim, min_dist, group_range)
    x = get_state(goal, q_people, t_people, p_people, tot_area, res)
    y = get_label(obs, tot_area, res)
             
    return [x,y]

#### Autogenerator

In [5]:
def data_generator(tot_area, res, people_range, p_per_group, area_dim, min_dist, group_range, total_items=20):
    """
    """
    i = 0
    while i < total_items:
        x,y = gather_single_data(tot_area, res, people_range, p_per_group, area_dim, min_dist, group_range)
        yield x, y
        i+=1

In [6]:
def my_input_fn(tot_area, res, people_range, p_per_group, area_dim, min_dist, group_range, total_items=20):
    dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(tot_area, res, people_range, p_per_group, area_dim, min_dist, group_range, total_items),
        output_signature=(
            tf.TensorSpec(shape=(120,120,3), dtype=tf.uint8),
            tf.TensorSpec(shape=(120,120,1), dtype=tf.uint8)
        )
    )
    dataset = dataset.batch(32)
    return dataset

In [7]:
def get_autogenerator_dataset(tot_area, res, people_range, p_per_group, area_dim, min_dist, group_range, total_items=20):
    """
    """
    dataset = my_input_fn(tot_area, res, people_range, p_per_group, area_dim, min_dist, group_range, total_items)

    return dataset